# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

import sys
# Import API key
# import api_keys
from api_keys import (g_key, weather_api_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_pull = "../WeatherPy/weather_df.csv"
weather_data_df = pd.read_csv(weather_data_pull)
weather_data_df = weather_data_df.dropna()
del weather_data_df["Unnamed: 0"]
weather_data_df.head(15)

,City,Country,Date,Latitude,Longitude,Cloudiness %,Humidity %,Max Temp (F),Wind Speed (mph)
0,provideniya,RU,2021-09-26 18:03:03,64.3833,-173.3000,40.0,75.0,37.778,7.00
1,niamey,NE,2021-09-26 18:00:14,13.5137,2.1098,59.0,32.0,91.778,3.28
2,asfi,MA,2021-09-26 18:02:25,32.2994,-9.2372,83.0,78.0,70.628,6.21
3,new norfolk,AU,2021-09-26 18:00:58,-42.7826,147.0587,24.0,76.0,48.020,0.45
4,isangel,VU,2021-09-26 18:03:06,-19.5500,169.2667,53.0,63.0,75.416,4.18
5,khatanga,RU,2021-09-26 18:01:11,71.9667,102.5000,91.0,92.0,28.418,3.85
6,aden,YE,2021-09-26 18:03:09,12.7794,45.0367,15.0,76.0,86.234,2.04
7,barrow,US,2021-09-26 18:03:10,71.2906,-156.7887,90.0,86.0,21.488,3.09
8,astaneh-ye ashrafiyeh,IR,2021-09-26 18:03:11,37.2596,49.9444,0.0,78.0,61.286,1.63
9,komsomolskiy,UZ,2021-09-26 18:03:12,40.4272,71.7189,0.0,26.0,67.190,2.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key= g_key)

In [4]:
coordinates = weather_data_df[['Latitude', 'Longitude']]
weights = weather_data_df['Humidity %']
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=weights, max_intensity=max(weather_data_df['Humidity %']))
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
filtered_data_df = weather_data_df
filtered_data_df = filtered_data_df.dropna()
filtered_data_df = filtered_data_df.loc[(filtered_data_df['Max Temp (F)'] < 80) & (filtered_data_df['Max Temp (F)'] > 70) & (filtered_data_df['Wind Speed (mph)'] < 10) & (filtered_data_df['Cloudiness %'] == 0)]
filtered_data_df.head()

,City,Country,Date,Latitude,Longitude,Cloudiness %,Humidity %,Max Temp (F),Wind Speed (mph)
69,porto santo,PT,2021-09-26 18:03:56,33.0720,-16.3415,0.0,73.0,71.780,1.03
163,gazi,GR,2021-09-26 18:04:44,35.3253,25.0669,0.0,72.0,72.878,0.00
168,jalu,LY,2021-09-26 18:04:45,29.0331,21.5482,0.0,44.0,73.634,4.77
239,fasa,IR,2021-09-26 18:05:01,28.9383,53.6482,0.0,18.0,72.554,1.16
287,san quintin,MX,2021-09-26 18:04:37,30.4833,-115.9500,0.0,66.0,71.564,5.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=filtered_data_df
hotel_df['Hotel Name']=''
hotel_df

,City,Country,Date,Latitude,Longitude,Cloudiness %,Humidity %,Max Temp (F),Wind Speed (mph),Hotel Name
69,porto santo,PT,2021-09-26 18:03:56,33.0720,-16.3415,0.0,73.0,71.780,1.03,
163,gazi,GR,2021-09-26 18:04:44,35.3253,25.0669,0.0,72.0,72.878,0.00,
168,jalu,LY,2021-09-26 18:04:45,29.0331,21.5482,0.0,44.0,73.634,4.77,
239,fasa,IR,2021-09-26 18:05:01,28.9383,53.6482,0.0,18.0,72.554,1.16,
287,san quintin,MX,2021-09-26 18:04:37,30.4833,-115.9500,0.0,66.0,71.564,5.67,
339,quelimane,MZ,2021-09-26 18:05:31,-17.8786,36.8883,0.0,100.0,73.616,2.06,
351,vathi,GR,2021-09-26 18:05:35,38.4000,23.6000,0.0,57.0,74.984,1.23,
355,ritchie,ZA,2021-09-26 18:05:36,-29.0380,24.6028,0.0,12.0,70.610,2.09,
378,mount isa,AU,2021-09-26 18:05:41,-20.7333,139.5000,0.0,17.0,73.436,2.57,
398,marsa matruh,EG,2021-09-26 18:05:48,31.3525,27.2453,0.0,63.0,73.022,2.62,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {'radius':5000, 'types': 'hotel', 'key': g_key}

for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params['location'] = f'{lat},{lng}'
    response = requests.get(base_url, params=params).json()

    results = response['results']
    try:
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['Name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df.head()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Date,Latitude,Longitude,Cloudiness %,Humidity %,Max Temp (F),Wind Speed (mph),Hotel Name
69,porto santo,PT,2021-09-26 18:03:56,33.0720,-16.3415,0.0,73.0,71.780,1.03,
163,gazi,GR,2021-09-26 18:04:44,35.3253,25.0669,0.0,72.0,72.878,0.00,
168,jalu,LY,2021-09-26 18:04:45,29.0331,21.5482,0.0,44.0,73.634,4.77,
239,fasa,IR,2021-09-26 18:05:01,28.9383,53.6482,0.0,18.0,72.554,1.16,
287,san quintin,MX,2021-09-26 18:04:37,30.4833,-115.9500,0.0,66.0,71.564,5.67,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))